<a href="https://colab.research.google.com/github/AmbarM2609/Gender_Voice_Recognition/blob/main/gender_voice_recognition_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download primaryobjects/voicegender

Dataset URL: https://www.kaggle.com/datasets/primaryobjects/voicegender
License(s): CC-BY-NC-SA-4.0
  0% 0.00/405k [00:00<?, ?B/s]
100% 405k/405k [00:00<00:00, 81.3MB/s]


In [ ]:
! unzip /content/voicegender.zip

Archive:  /content/voicegender.zip
replace voice.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**What is Logistic Regression?**

* Logistic Regression is used in the classification of numerical or catagorical data. The result of the dependent variable can only have 2 different values, for example yes / no, 1/0, female / male.
* Now, let's use logistic regression, one of scikit-learn's popular machine learning algorithms, to train our diabetes dataset.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/voicegender/voice.csv


**About Dataset**

**Voice Gender**

Gender Recognition by Voice and Speech Analysis

This database was created to identify a voice as male or female, based upon acoustic properties of the voice and speech. The dataset consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by acoustic analysis in R using the seewave and tuneR packages, with an analyzed frequency range of 0hz-280hz (human vocal range).

In [ ]:
#%% libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read csv and drop
df = pd.read_csv("/kaggle/input/voicegender/voice.csv")
df.head(10)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male
5,0.132786,0.079557,0.119090,0.067958,0.209592,0.141634,1.932562,8.308895,0.963181,0.738307,...,0.132786,0.110132,0.017112,0.253968,0.298222,0.007812,2.726562,2.718750,0.125160,male
6,0.150762,0.074463,0.160106,0.092899,0.205718,0.112819,1.530643,5.987498,0.967573,0.762638,...,0.150762,0.105945,0.026230,0.266667,0.479620,0.007812,5.312500,5.304688,0.123992,male
7,0.160514,0.076767,0.144337,0.110532,0.231962,0.121430,1.397156,4.766611,0.959255,0.719858,...,0.160514,0.093052,0.017758,0.144144,0.301339,0.007812,0.539062,0.531250,0.283937,male
8,0.142239,0.078018,0.138587,0.088206,0.208587,0.120381,1.099746,4.070284,0.970723,0.770992,...,0.142239,0.096729,0.017957,0.250000,0.336476,0.007812,2.164062,2.156250,0.148272,male
9,0.134329,0.080350,0.121451,0.075580,0.201957,0.126377,1.190368,4.787310,0.975246,0.804505,...,0.134329,0.105881,0.019300,0.262295,0.340365,0.015625,4.695312,4.679688,0.089920,male


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

Our **label** feature dtype is object, so we couldn't use at categorical classification. In this reason, we have to change to ones or zeros.

In [ ]:
df.label=[1 if each=='male' else 0 for each in df.label]

In [ ]:
y = df.label.values
x = df.drop(["label"],axis=1)

We will use the sklearn library to train voices data. By importing train_test_split from sklearn, we sort voices data randomly and assign it to variables like x_train, x_test, y_train, y_test. Here; x_train and y_train; variables to use for machine learning, x_test and y_test; the variables required to find out the success percentage of the code we wrote.

In [ ]:
#division of data for training and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42 )

Data distribution: Whether data is distributed normally is a factor that affects the operation of some algorithms. Model performance may be affected if the data is right or left tilted. We will use StandardScaler to avoid this.

In [ ]:
#scailing of data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [ ]:
#training
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)
print(y_pred)
print("Logistic regressin score: {}".format(lr.score(X_train,y_train)))

[0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1
 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 1
 1 0 1 1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 0 0
 0 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 0 0 0
 0 1 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1 1 1 0
 0 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1
 0 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 1
 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 0
 1 0 0 0 1 0 0 1 1 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0
 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 1
 1 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1
 1 0 0 0 1 1 0 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1 0 1 0 1 1 1
 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 0 0 0 1
 0 1 1 0 0 1 1 1 0 1 0 1 